In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
import sys

In [3]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
from posarmctools.readdata import *

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

In [6]:
data_date = "2018_06_27_13_18_13"
data_dir = "/home/pleroy/DATA/2018_06_27_LETG/2018_06_27/jde/" + data_date

In [7]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters_v2( params_filename )
params.print()

requestedTRamp 1200.0
configuredTRamp 1200.0
startFrequency 5725000000.0
stopFrequency 5875000000.0
frequencyBand 150000000.0
frequencyDevReq 50000.0
frequencyDevConf 50001.1
numberOfSteps 3000
waveformType 1
rampsPerBuffer 375
bufferSize 9000000.0
buffersPerFile 2
rampsPerFile 750
fileSize 18000000.0
samplingFrequency 10000000.0
samplesPerRamp 12000
skipNSamples 0


In [8]:
buffersPerFile = params.buffersPerFile
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
samplesPerFile = samplesPerRamp * rampsPerFile

print( "buffersPerFile = {}, samplesPerRamp = {}, rampsPerFile = {}".format(
    buffersPerFile, samplesPerRamp, rampsPerFile )
     )

# selection in record
firstFile = 0
nbFiles = 325
lastFile = firstFile + nbFiles - 1
firstBuffer = firstFile * buffersPerFile

buffersPerFile = 2, samplesPerRamp = 12000, rampsPerFile = 750


# Read the data

In [9]:
scalingFactor = 2 / 65535
offset = -32768

In [10]:
timeSerie_A = np.zeros( samplesPerFile )
adc_A = np.zeros( (nbFiles, samplesPerFile) )

In [11]:
lastBuffer = (nbFiles-1) * buffersPerFile + firstBuffer
for k in range( nbFiles ):
    
    # read the data
    nb = str( int( k * buffersPerFile + firstBuffer) )
    filename = data_dir + "/record" + nb + ".bin"
    readFileADLINKCh0( filename, samplesPerFile, timeSerie_A )
    
    print("k = {}, block = {} / {}".format(k, nb, lastBuffer))
    
    adc_A[ k, : ] = (timeSerie_A + offset) * scalingFactor

k = 0, block = 0 / 648
k = 1, block = 2 / 648
k = 2, block = 4 / 648
k = 3, block = 6 / 648
k = 4, block = 8 / 648
k = 5, block = 10 / 648
k = 6, block = 12 / 648
k = 7, block = 14 / 648
k = 8, block = 16 / 648
k = 9, block = 18 / 648
k = 10, block = 20 / 648
k = 11, block = 22 / 648
k = 12, block = 24 / 648
k = 13, block = 26 / 648
k = 14, block = 28 / 648
k = 15, block = 30 / 648
k = 16, block = 32 / 648
k = 17, block = 34 / 648
k = 18, block = 36 / 648
k = 19, block = 38 / 648
k = 20, block = 40 / 648
k = 21, block = 42 / 648
k = 22, block = 44 / 648
k = 23, block = 46 / 648
k = 24, block = 48 / 648
k = 25, block = 50 / 648
k = 26, block = 52 / 648
k = 27, block = 54 / 648
k = 28, block = 56 / 648
k = 29, block = 58 / 648
k = 30, block = 60 / 648
k = 31, block = 62 / 648
k = 32, block = 64 / 648
k = 33, block = 66 / 648
k = 34, block = 68 / 648
k = 35, block = 70 / 648
k = 36, block = 72 / 648
k = 37, block = 74 / 648
k = 38, block = 76 / 648
k = 39, block = 78 / 648
k = 40, block =

k = 310, block = 620 / 648
k = 311, block = 622 / 648
k = 312, block = 624 / 648
k = 313, block = 626 / 648
k = 314, block = 628 / 648
k = 315, block = 630 / 648
k = 316, block = 632 / 648
k = 317, block = 634 / 648
k = 318, block = 636 / 648
k = 319, block = 638 / 648
k = 320, block = 640 / 648
k = 321, block = 642 / 648
k = 322, block = 644 / 648
k = 323, block = 646 / 648
k = 324, block = 648 / 648


In [12]:
A_reshaped = adc_A.reshape(nbFiles * rampsPerFile, samplesPerRamp)

**Check that the first ramp is an up ramp**

In [13]:
plt.figure()
plt.plot( A_reshaped[0, :] )
title = "first acquisition " + data_date
plt.title(title)
plt.grid()
plt.savefig( data_dir + "/" + title + ".png", bbox_inches='tight')

**Set the shifted flag properly**

In [14]:
rampDownFirst = 1

# Build RD

In [15]:
#RDc = build_rd_from_data( params, A_reshaped, shifted )
withHanning = 1
RDc = build_rd_from_data_rampDown( params, A_reshaped, rampDownFirst, withHanning )

shape of the samples matrix = (243750, 12000)
with Hanning window
ramp down first in the data files


In [16]:
coupling_RD = np.average(RDc, 0)
if withHanning:
    np.save( data_dir + "/coupling_RD_files_{}_{}_rampDown_hanning".format(firstFile, lastFile), coupling_RD )
else:
    np.save( data_dir + "/coupling_RD_files_{}_{}_rampDown".format(firstFile, lastFile), coupling_RD )

In [17]:
plt.figure()
plt.plot(np.abs(coupling_RD), 'k', label='coupling')
plt.legend()
plt.grid()

In [ ]:
if withHanning:
    np.save(data_dir + '/RD_files_{}_{}_rampDown_hanning'.format(firstFile, lastFile), RDc) 
else:
    np.save(data_dir + '/RD_files_{}_{}_rampDown'.format(firstFile, lastFile), RDc)